## Setting up exchange-only experiments

### Introduction
In this tutorial we will demonstrate how to use spinqick to calibrate an axis of rotation on an exchange-only qubit.  Spinqick is set up to use the firmware which implements crosstalk compensation in DSP, so you'll need to make sure that is loaded.

In [ ]:
import Pyro4
from qick import QickConfig
from spinqick.settings import file_settings
from spinqick.models import qubit_models, full_experiment_model
from spinqick.helper_functions import file_manager, hardware_manager
from spinqick.experiments import eo_single_qubit

In [ ]:
Pyro4.config.SERIALIZER = "pickle"
Pyro4.config.PICKLE_PROTOCOL_VERSION = 4

ns_host = "192.168.2.99"  # make sure this matches your board's ip address!
ns_port = 8888
proxy_name = "myqick"

ns = Pyro4.locateNS(host=ns_host, port=ns_port)
soc = Pyro4.Proxy(ns.lookup(proxy_name))
soccfg = QickConfig(soc.get_cfg())
print(soccfg)

### Setting up your experiment config

Let's start by editing the config we've been working with.  We need to start by defining each exchange axis of the qubit.  We will need to define pulse and idle times, and the gates associated with each gate. 


In [ ]:
# load the current config
full = file_manager.load_config_json(
    file_settings.dot_experiment_config, full_experiment_model.ExperimentConfig
)

# Defining the n-axis.  This requires us to define the neighboring p-gates (denoted px and py) and the x-gate associated with this axis.
# The model we use for this also takes the idle and exchange voltage coordinates, but for the experiments in spinqick currently,
# the exchange coordinates on the p-gates are not used.
n_gates = qubit_models.ExchangeGateMapParams(
    px=qubit_models.ExchangeGateParams(
        name="P2",
        gate_voltages=qubit_models.ExchangeVoltages(idle_voltage=0, exchange_voltage=0),
    ),  # px vs py is somewhat arbitrary, just pick the p-gate you want plotted on the x- or y-axis
    py=qubit_models.ExchangeGateParams(
        name="P3",
        gate_voltages=qubit_models.ExchangeVoltages(idle_voltage=0, exchange_voltage=0),
    ),
    x=qubit_models.ExchangeGateParams(
        name="X2",
        gate_voltages=qubit_models.ExchangeVoltages(
            idle_voltage=0, exchange_voltage=0.1
        ),
    ),
)
# We plug these parameters into a model which contains information about the idlde and exchange times, and the exchange and deturning vectors.
# With the crosstalk compensation firmware we don't need to worry about these, since we can take care of them by setting crosstalk registers.
n = qubit_models.ExchangeAxisConfig(
    gates=n_gates,
    times=qubit_models.ExchangeTimes(idle_time=0.02, exchange_time=0.01),
    detuning_vector=[1, -1, 0],
    exchange_vector=[0, 0, 1],
    symmetric_vector=[0, 0, 1],
)

# repeat for z
z_gates = qubit_models.ExchangeGateMapParams(
    px=qubit_models.ExchangeGateParams(
        name="P1",
        gate_voltages=qubit_models.ExchangeVoltages(idle_voltage=0, exchange_voltage=0),
    ),
    py=qubit_models.ExchangeGateParams(
        name="P2",
        gate_voltages=qubit_models.ExchangeVoltages(idle_voltage=0, exchange_voltage=0),
    ),
    x=qubit_models.ExchangeGateParams(
        name="X1",
        gate_voltages=qubit_models.ExchangeVoltages(
            idle_voltage=0, exchange_voltage=0.1
        ),
    ),
)
z = qubit_models.ExchangeAxisConfig(
    gates=z_gates,
    times=qubit_models.ExchangeTimes(idle_time=0.02, exchange_time=0.01),
    detuning_vector=[1, -1, 0],
    exchange_vector=[0, 0, 1],
    symmetric_vector=[0, 0, 1],
)

# define the full qubit
qubit = qubit_models.Eo1QubitAxes(z=z, n=n)
# add in our pauli spin blockade readout parameters
q1 = full_experiment_model.QubitParams(
    ro_cfg=full.qubit_configs["q1"].ro_cfg, qubit_params=qubit
)
full_2 = full.model_copy()
full_2.qubit_configs["q1"] = q1
### save this config if desired
file_manager.save_config_json(full_2, file_settings.dot_experiment_config)
### to print config as a dictionary:
full_2.model_dump()

Once again, we can initialize our DC source and the `EOSingleQubit` experiment class.

In [ ]:
vdc = hardware_manager.DummyDCSource()
eo = eo_single_qubit.EOSingleQubit(soccfg, soc, vdc, save_data=False, plot=False)

### Nonequilibrium cell

We start by turning on exchange and scanning the (1,1,1) charge cell.  This will sweep voltages of the `px` and `py` gates associated with the axis you select (in our case P1 and P2 as defined in the `n_gates` parameters defined above).  The `exchange_voltage` and `exchange_time` parameters in your experiment config define the duration and amplitude of this pulse. If your compensation matrix is correct, the charge cell will not move as you change the exchange pulse amplitude.

In [ ]:
data = eo.do_nonequilibrium(
    qubit="q1", axis="n", p_range = ((-0.1, 0.1, 100), (-0.1, 0.1, 100)), point_avgs=2, full_avgs=2
)

![NonequilibriumCell](demo_data/1760730254_noneq_cell.png)

### Fingerprint

Once we've verified that our compensation matrix is working reasonably well, we can look for a fingerprint.  Before executing a fingerprint or nonequilibrium cell on the z-axis, set the `exchange_voltage` parameter on the n-axis to approximately a pi rotation. The code adds a pulse before and after each shot on the n-axis.

In [ ]:
data = eo.do_fingerprint(
    qubit="q1", axis="n", detuning_range=(-0.05, 0.05, 100), exchange_range=(0, 0.1, 100), point_avgs=2, full_avgs=2
)

![Fingerprint](demo_data/1761593907_fingerprint.png)

### Exchange angle calibrations

SpinQICK contains two built-in calibration functions.  `course_cal` performs a sweep linearly in voltage space, and `fine_cal` uses the course calibration to calculate voltages spaced linearly in phase.  The parameters can be entered into the `exchange_cal` field of your `dot_experiment_config` for that axis.

In [ ]:
cc = eo.course_cal("q1", "n", (0.1, 0.15, 400), fit=False)
# if fit is set to true and is successful, fit parameters are stored here:
cc.best_fit

![CourseCal](demo_data/1760728765_coursecal.png)

Now that we have executed the calibration, we need to add this data to our config file. We add the `A`, `B` and `theta_max` parameters from the fit to our config.

In [ ]:
full = file_manager.load_config_json(
    file_settings.dot_experiment_config, full_experiment_model.ExperimentConfig
)
full.qubit_configs["q1"].qubit_params.n.exchange_cal = qubit_models.ExchangeCalibration(
    rough_num_pulses=3,
    fine_num_pulses=10,
    cal_parameters=qubit_models.CalParameters(
        A=0.1, B=0.1, theta_max=10, volt_list=[], theta_list=[]
    ),
)
file_manager.save_config_json(full, file_settings.dot_experiment_config)

We can perform a fine calibration now.  We will need to copy the list of calibrated values from the output of the fine calibration function to our config file.

In [ ]:
fc = eo.fine_cal("q1", "n", (0.01, 7, 100), t_res="fs", point_avgs=100, full_avgs=1)
theta_list = fc.analyzed_data
volt_list = fc.dset_coordinates

![Finecal_1](demo_data/1761160541_finecal.png)
![Finecal_2](demo_data/1761160541_finecal_2.png)

Once we've copied those values over, we have a calibrated axis.  We can go back and repeat these steps on the z-axis. Now we have a qubit!